In [4]:
import pandas as pd
from sklearn.linear_model import LinearRegression, LogisticRegression
from matplotlib import pyplot as plt
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, MinMaxScaler, RobustScaler, Normalizer
from sklearn.model_selection import train_test_split
from imblearn.over_sampling import RandomOverSampler
from sklearn.svm import SVC
from collections import Counter
from sklearn.metrics import accuracy_score
import joblib

In [5]:
# read in interest rate csv file
interest = pd.read_csv("Clean_Data/clean_fedfunds.csv")
# change data types from int to string
interest["Month"] = interest["Month"].astype(str)
interest["Year"] = interest["Year"].astype(str)
# filter from 2002
interest = interest.loc[interest["Year"] >= "2002"]
interest.head()

,Year,Month,FEDFUNDS
0,2002,1,1.73
1,2002,2,1.74
2,2002,3,1.73
3,2002,4,1.75
4,2002,5,1.75


In [6]:
# read in industrial output csv file
indus = pd.read_csv("Clean_Data/Industrial.csv")
#change data types from int to string
indus["Month"] = indus["Month"].astype(str)
indus["Year"] = indus["Year"].astype(str)
indus.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Clean_Data/Industrial.csv'

In [7]:
# read in Retail Sales data 
sales = pd.read_csv("Clean_Data/Retail_Sales.csv")
# change out int to string
sales["Month"] = sales["Month"].astype(str)
sales["Year"] = sales["Year"].astype(str)
sales.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Clean_Data/Retail_Sales.csv'

In [8]:
# read in inflation data
inflation = pd.read_csv("Clean_Data/inflation.csv")
#change to string
inflation["Year"] = inflation["Year"].astype(str)
inflation.head()

FileNotFoundError: [Errno 2] No such file or directory: 'Clean_Data/inflation.csv'

In [753]:
# read in unemployment data
unemployment = pd.read_csv("Clean_Data/Unemployment.csv")
#change int to string
unemployment["Year"] = unemployment["Year"].astype(str)
unemployment.head()

,Year,1,2,3,4,5,6,7,8,9,10,11,12
0,2002,5.7,5.7,5.7,5.9,5.8,5.8,5.8,5.7,5.7,5.7,5.9,6.0
1,2003,5.8,5.9,5.9,6.0,6.1,6.3,6.2,6.1,6.1,6.0,5.8,5.7
2,2004,5.7,5.6,5.8,5.6,5.6,5.6,5.5,5.4,5.4,5.5,5.4,5.4
3,2005,5.3,5.4,5.2,5.2,5.1,5.0,5.0,4.9,5.0,5.0,5.0,4.9
4,2006,4.7,4.8,4.7,4.7,4.6,4.6,4.7,4.7,4.5,4.4,4.5,4.4


In [2479]:
# create empty dataframe
data = pd.DataFrame()
ticker_list = ["AAPL", "NVDA", "AMZN", "TSLA", "MSFT", "GOOG", "META", "GOOGL", "PEP", "COST", "AVGO", "ADBE", "CSCO", "CMCSA", "TMUS", "INTC", "TXN", "AMD", "QCOM", "AMGN", "HON", "INTU", "FISV", "MRNA", "PANW", "KDP", "KLAC", "MNST", "SNPS", "AEP", "FTNT", "KHC", "MAR", "PAYX", "ASML", "CDNS", "EXC", "ORLY", "ADSK"]
#read in tickers
for i in ticker_list:
    temp = pd.read_csv(f"Extracting_Data/{i}/{i}_monthly_1_False.csv")
    # add ticker name to each row
    temp["Ticker"] = i
    # combines each ticker csv on top of eachother
    data = pd.concat([data, temp])
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker
0,2002-06-30 22:00:00,0.316250,0.272500,0.335536,0.246429,8118415200,-13.833992,AAPL
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL


In [2480]:


#add a future column where shows the gain and add column to see where tickers overlap
data['future'] = data["Gain%"].shift(-1)
data["Mov_Ticker"] = data["Ticker"].shift(-1)
data = data.dropna()

#drop rows where ticker info overlaps
data = data.loc[data["Mov_Ticker"] == data["Ticker"]]
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,Mov_Ticker
0,2002-06-30 22:00:00,0.316250,0.272500,0.335536,0.246429,8118415200,-13.833992,AAPL,-2.382529,AAPL
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL,0.069014,AAPL
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,AAPL
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,AAPL
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,AAPL


In [2481]:
R_g = []
for x in data["Gain%"]:
    if x <= 0:
        R_g.append(0)
    if x > 0:
        R_g.append(1)
    #add column for calculation
data["Current_R/G"] = R_g

In [2482]:
def data_transform(data):
    #create arrays
    inflation_stat = []
    unemployment_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:7]

        # locates in inflation/unemployment data when the year is the same as the current itteration
        inf_month = inflation.loc[inflation["Year"] == [i][0][:4]]
        unemp_month = unemployment.loc[unemployment["Year"] == [i][0][:4]]
        
        #locates the month of the filtered data and extracts the value
        inflation_stat.append(inf_month[month].values[0])
        unemployment_stat.append(unemp_month[month].values[0])

    # adds arrays to dataset
    data["Inflation"] = inflation_stat
    data["Unemployment"] = unemployment_stat
    # eliminates month from data
    data = data.loc[data["Datetime"] < "2022-06"]
    
    # create arrays
    industrial_stat = []
    interest_stat = []
    sales_stat = []
    #iterate through each datetime in the dataset
    for i in data["Datetime"]:
        #calculates if the month is formatted as 0# and assigns month to number #
        if i[5] == "0":
            month = i[6]
        # if formatted as ## then month assigned this number ##
        else:
            month = i[5:6]
        
        # filters datasets with the year in the iteration and when the month of the dataset is = "month"
        row = interest.loc[(interest["Year"] == [i][0][:4]) & (interest["Month"] == month)]
        ind = indus.loc[(indus["Year"] == [i][0][:4]) & (indus["Month"] == month)]
        sal = sales.loc[(sales["Year"] == [i][0][:4]) & (sales["Month"] == month)]
        
        # append stats to arrays
        interest_stat.append(row["FEDFUNDS"].values[0])
        industrial_stat.append(ind["INDPRO"].values[0])
        sales_stat.append(sal["RSXFS"].values[0])
    # adds arrays to dataset
    data["Industrial"] = industrial_stat
    data["Interest"] = interest_stat
    data["Retail_Sales"] = sales_stat
    
    # add new columns with percent changed
    data["Interest_pct"] = data["Interest"].pct_change()
    data["Inflation_pct"] = data["Inflation"].pct_change()
    data["Unemployment_pct"] = data["Unemployment"].pct_change()
    data["Industrial_pct"] = data["Industrial"].pct_change()
    data["Volume_pct"] = data["Volume"].pct_change()
    data["Retail_Sales_pct"] = data["Retail_Sales"].pct_change() 
    data = data.dropna()

    # calculates if the future gain is positive or negative
    R_g = []
    for x in data["future"]:
        if x <= 0:
            R_g.append(0)
        if x > 0:
            R_g.append(1)
    #add column for calculation
    data["R/G"] = R_g
    
    return data


In [2483]:
# call function
data = data_transform(data)
data.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,Ticker,future,Mov_Ticker,...,Industrial,Interest,Retail_Sales,Interest_pct,Inflation_pct,Unemployment_pct,Industrial_pct,Volume_pct,Retail_Sales_pct,R/G
1,2002-07-31 22:00:00,0.269821,0.263393,0.290179,0.249464,5151686400,-2.382529,AAPL,0.069014,AAPL,...,90.8761,1.73,262769,-0.011429,-0.043478,0.000000,-0.000349,-0.365432,0.011483,1
2,2002-08-31 22:00:00,0.258750,0.258929,0.271250,0.250893,5291837600,0.069014,AAPL,10.143926,AAPL,...,90.7850,1.74,265043,0.005780,0.090909,-0.017241,-0.001002,0.027205,0.008654,1
3,2002-09-30 22:00:00,0.260536,0.286964,0.293571,0.238571,6611432800,10.143926,AAPL,-2.760345,AAPL,...,90.8931,1.75,260626,0.005747,-0.083333,0.000000,0.001191,0.249364,-0.016665,0
4,2002-10-31 22:00:00,0.284643,0.276786,0.310357,0.268036,4531721600,-2.760345,AAPL,-9.874217,AAPL,...,88.6706,1.73,256307,-0.011429,0.000000,0.000000,-0.024452,-0.314563,-0.016572,0
5,2002-11-30 22:00:00,0.283929,0.255893,0.287500,0.246071,4530439200,-9.874217,AAPL,0.000000,AAPL,...,88.6706,1.73,256307,0.000000,-0.090909,0.035088,0.000000,-0.000283,0.000000,0


In [2484]:
#index to test/predict for
tester = pd.read_csv("Extracting_Data/QQQ/QQQ_monthly_1_False.csv")
#add a future column where shows the gain for next row
tester['future'] = tester["Gain%"].shift(-1)
tester = tester.dropna()
tester.head()

,Datetime,Open,Close,High,Low,Volume,Gain%,future
0,2002-06-30 22:00:00,25.97,23.85,26.55,21.64,2668049100,-8.163265,-1.053075
1,2002-07-31 22:00:00,23.74,23.49,26.21,21.30,2043810216,-1.053075,-10.147441
2,2002-08-31 22:00:00,23.06,20.72,24.35,20.49,1669047392,-10.147441,17.407939
3,2002-09-30 22:00:00,20.91,24.55,25.04,19.76,2129150120,17.407939,13.746410
4,2002-10-31 22:00:00,24.37,27.72,28.29,24.14,1669954692,13.746410,-14.250528


In [2485]:
R_g = []
for x in tester["Gain%"]:
    if x <= 0:
        R_g.append(0)
    if x > 0:
        R_g.append(1)
    #add column for calculation
tester["Current_R/G"] = R_g

In [3]:
tester = data_transform(tester)
tester.tail()

NameError: name 'data_transform' is not defined

In [2911]:
### model for stock tickers with only Open, Close, Volume

X = data[["Open", "Close", "Volume"]]
y = data["R/G"]

#columns from index to test
TestX = tester[["Open", "Close", "Volume"]]
Testy = tester["R/G"]

#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

accuracy_score(y_test, y_pred)

0.5205214872042492

In [2872]:
## Model with macro economic data 

X = data[[ "Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct", "Open", "Close", "Volume_pct", "Volume"]]
y = data ["R/G"]

#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

accuracy_score(y_test, y_pred)


0.680830516658619

In [2]:
#create sample of tester data to test accuracy of model 
testerSample = tester.sample(n=30)

#get date of sample
sample_list = testerSample["Datetime"].tolist()

#copy datafrane
tester_data = testerSample.copy()

#get data where it does not have these dates
for date in sample_list:
    tester_data = tester_data.loc[testerme["Datetime"] != date]

NameError: name 'tester' is not defined

In [ ]:
#columns to train model for stock tickers
X = tester_data[[ "Interest_pct","Unemployment_pct", "Industrial_pct", "Retail_Sales_pct","Inflation_pct", "Open", "Close", "Volume_pct", "Volume"]]
y = tester_data["R/G"]

#columns from index to test
TestX = testerSample[[ "Interest_pct", "Unemployment_pct", "Industrial_pct", "Retail_Sales_pct", "Inflation_pct", "Open", "Close", "Volume_pct", "Volume"]]
Testy = testerSample["R/G"]

In [2863]:
#train test and split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)
# Standardize data set
scaler = StandardScaler()
# Fit X_train data
scaler.fit(X_train)
#Transform scaled data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


# implement random oversampling
from imblearn.over_sampling import RandomOverSampler
ros = RandomOverSampler(random_state=1)
X_resampled, y_resampled = ros.fit_resample(X_train_scaled, y_train)

# Create a random forest classifier.
rf_model = RandomForestClassifier(n_estimators=128)

# Fitting the model
rf_model = rf_model.fit(X_resampled, y_resampled)

# Evaluate the model
y_pred = rf_model.predict(X_test_scaled)

accuracy_score(y_test, y_pred)

0.6856353591160221

In [2864]:
# fit tester sample data
scaled = scaler.fit_transform(TestX)
# Evaluate the model
y_pred = rf_model.predict(scaled)
# find accuracy of this sample data
accuracy_score(Testy, y_pred)

0.7

In [2810]:
# prediction for tester sample
pd.DataFrame({"Prediction": y_pred, "Actual": Testy})

,Prediction,Actual
71,0,1
115,1,1
137,1,0
54,0,0
217,0,0
62,0,1
153,1,1
161,1,0
80,1,1
181,0,0


In [2812]:
# Display the confusion matrix
from sklearn.metrics import confusion_matrix

confusion_matrix(Testy, y_pred)

array([[8, 6],
       [9, 7]])

In [2813]:
# Print the imbalanced classification report
from imblearn.metrics import classification_report_imbalanced

print(classification_report_imbalanced(Testy, y_pred))

                   pre       rec       spe        f1       geo       iba       sup

          0       0.47      0.57      0.44      0.52      0.50      0.25        14
          1       0.54      0.44      0.57      0.48      0.50      0.25        16

avg / total       0.51      0.50      0.51      0.50      0.50      0.25        30



In [2925]:

# We can sort the features by their importance.
sorted(zip(rf_model.feature_importances_, X.columns), reverse=True)

[(0.32936000810266297, 'Volume'),
 (0.32835497145085357, 'Volume_pct'),
 (0.038652005831717885, 'Unemployment_pct'),
 (0.037494351359503235, 'Industrial_pct'),
 (0.03594246347726869, 'Industrial'),
 (0.03474390385577661, 'Retail_Sales_pct'),
 (0.03395097688955526, 'Interest'),
 (0.03328308742123191, 'Retail_Sales'),
 (0.033281935593225107, 'Interest_pct'),
 (0.03261692886566243, 'Inflation_pct'),
 (0.03172440521205474, 'Unemployment'),
 (0.030594961940487622, 'Inflation')]

In [2474]:
# change type of dates
date = []
for i in data["Datetime"]:
    date.append(i[0:10])
data["Datetime"] = date    

In [2475]:
# exporting to csv
data.to_csv("Tech_Stocks.csv")

In [ ]:
# save model 
joblib.dump(rf_model, 'practice_machine_learning.joblib')